## Finetune Flair

In [2]:
import os
from pathlib import Path
from sklearn.model_selection import train_test_split
import pandas as pd

import torch

from flair.data import Corpus, Sentence
from flair.datasets import ClassificationCorpus
from flair.embeddings import (
    TransformerDocumentEmbeddings,
    FlairEmbeddings, 
    DocumentRNNEmbeddings
)
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from flair.training_utils import EvaluationMetric
from flair.visual.training_curves import Plotter

import tqdm

from torch.optim import AdamW
import logging

import flair
flair.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {flair.device}")

from collections import Counter

from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score

Using device: cuda


In [3]:
import torch
print("PyTorch Version:", torch.__version__)
print("CUDA Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA Version:", torch.version.cuda)

PyTorch Version: 2.2.1+cu121
CUDA Available: True
CUDA Version: 12.1


In [4]:
# Ensure the output directory exists
flair_data_folder = "data/flair_data"
os.makedirs(flair_data_folder, exist_ok=True)

# Define the folder where the data is located
corpus_folder = Path(flair_data_folder)

# Create the corpus
corpus = ClassificationCorpus(
    corpus_folder,
    train_file='train.txt',
    dev_file='dev.txt',
    test_file='test.txt',
    label_type='sentiment'
)

# Print statistics
print(f"Number of training sentences: {len(corpus.train)}")
print(f"Number of validation sentences: {len(corpus.dev)}")
print(f"Number of test sentences: {len(corpus.test)}")

# Small dataset
small_data_folder = "data/flair_data"
small_corpus = ClassificationCorpus(small_data_folder, 
                                    train_file="train_small.txt",  
                                    dev_file='dev_small.txt', 
                                    test_file='test_small.txt',
                                    label_type="sentiment")

# Print statistics
print(f"Number of training sentences: {len(small_corpus.train)}")
print(f"Number of validation sentences: {len(small_corpus.dev)}")
print(f"Number of test sentences: {len(small_corpus.test)}")


2024-12-05 11:42:53,689 Reading data from data\flair_data
2024-12-05 11:42:53,689 Train: data\flair_data\train.txt
2024-12-05 11:42:53,689 Dev: data\flair_data\dev.txt
2024-12-05 11:42:53,690 Test: data\flair_data\test.txt
2024-12-05 11:42:54,355 Initialized corpus data\flair_data (label type name is 'sentiment')
Number of training sentences: 22500
Number of validation sentences: 2500
Number of test sentences: 25000
2024-12-05 11:42:54,356 Reading data from data\flair_data
2024-12-05 11:42:54,356 Train: data\flair_data\train_small.txt
2024-12-05 11:42:54,356 Dev: data\flair_data\dev_small.txt
2024-12-05 11:42:54,357 Test: data\flair_data\test_small.txt
2024-12-05 11:42:54,359 Initialized corpus data/flair_data (label type name is 'sentiment')
Number of training sentences: 55
Number of validation sentences: 25
Number of test sentences: 48


In [5]:
label_dict = corpus.make_label_dictionary(label_type='sentiment')
print(label_dict)

2024-12-05 11:42:54,515 Computing label dictionary. Progress:


0it [00:00, ?it/s]
22500it [00:55, 407.80it/s]

2024-12-05 11:43:49,694 Dictionary created for label 'sentiment' with 2 values: POS (seen 11298 times), NEG (seen 11202 times)
Dictionary with 2 tags: POS, NEG


In [6]:
def evaluate_model(classifier, test_dataset):
    """
    Evaluate a Flair classifier on a given test dataset, with verification and debugging steps.

    Args:
        classifier (TextClassifier): The trained Flair classifier.
        test_dataset (Dataset): The test dataset.

    Returns:
        dict: Dictionary containing evaluation metrics.
    """
    true_labels = []
    predicted_labels = []

    # Label mapping
    label_mapping = {"NEG": 0, "POS": 1}

    # Iterate over the test dataset with tqdm progress bar
    for sentence in tqdm.tqdm(test_dataset, desc="Evaluating", leave=True):
        sentence.to(flair.device)

        # Get true label
        true_label = sentence.get_label("sentiment").value
        true_labels.append(true_label)

        # Get predicted label
        classifier.predict(sentence)
        predicted_label = sentence.labels[0].value
        predicted_labels.append(predicted_label)

    # Verify label consistency
    print("True Labels Sample:", true_labels[:5])
    print("Predicted Labels Sample:", predicted_labels[:5])
    print("True Label Distribution:", Counter(true_labels))
    print("Predicted Label Distribution:", Counter(predicted_labels))

    # Map labels to numeric values for sklearn
    try:
        true_labels_mapped = [label_mapping[label] for label in true_labels]
        predicted_labels_mapped = [label_mapping[label] for label in predicted_labels]
    except KeyError as e:
        print(f"Label mapping error: {e}. Ensure all labels are in {label_mapping}.")
        return {}

    # Verify mapped labels
    print("Mapped True Labels Sample:", true_labels_mapped[:5])
    print("Mapped Predicted Labels Sample:", predicted_labels_mapped[:5])

    # Calculate metrics
    accuracy = accuracy_score(true_labels_mapped, predicted_labels_mapped)
    precision = precision_score(true_labels_mapped, predicted_labels_mapped, pos_label=1, zero_division=0)
    recall = recall_score(true_labels_mapped, predicted_labels_mapped, pos_label=1, zero_division=0)
    f1 = f1_score(true_labels_mapped, predicted_labels_mapped, pos_label=1, zero_division=0)

    # Full classification report
    classification_rep = classification_report(
        true_labels_mapped,
        predicted_labels_mapped,
        target_names=["NEG", "POS"]  # Match target names with label_mapping
    )

    # Print metrics
    print(f"\nAccuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print("\nClassification Report:")
    print(classification_rep)

    # Return metrics as a dictionary
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "classification_report": classification_rep,
    }

## flair embedding: to evaluate if code is working on small dataset

In [11]:
# Step 2: Initialize Flair embeddings
forward_embedding = FlairEmbeddings("news-forward")
backward_embedding = FlairEmbeddings("news-backward")

# Step 3: Create DocumentRNNEmbeddings using only Flair embeddings
document_embeddings = DocumentRNNEmbeddings(
    embeddings=[forward_embedding, backward_embedding],
    hidden_size=256,  # Adjust based on your computational resources
    reproject_words=True,  # Reproject word embeddings into a new space
    reproject_words_dimension=256,  # Dimensionality of reprojection
)

flair_classifier = TextClassifier(
    embeddings=document_embeddings,
    label_dictionary=label_dict,
    label_type="sentiment",
)


In [8]:
# Set the logging level to INFO
logging.basicConfig(level=logging.INFO)

# Initialize the trainer
trainer = ModelTrainer(flair_classifier, small_corpus)

#Fine-tune the model
trainer.fine_tune(
    base_path='flair_flair_small_test_model',             # Directory to save the model and logs
    learning_rate=5e-5,                  # Learning rate for fine-tuning
    mini_batch_size=8,                   # Smaller batch size for transformers
    max_epochs=3,                        # Number of epochs
    embeddings_storage_mode='none',     
    optimizer=AdamW,                     # Optimizer suited for transformers
    save_final_model=True,               # Save the final model
    save_model_each_k_epochs=1,          # Save model checkpoint every epoch
    create_file_logs=True,               # Save logs to a file
    create_loss_file=True,               # Save loss values to a file
    use_final_model_for_eval=False
)


2024-12-04 18:53:56,071 ----------------------------------------------------------------------------------------------------
2024-12-04 18:53:56,071 Model: "TextClassifier(
  (embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.05, inplace=False)
          (encoder): Embedding(300, 100)
          (rnn): LSTM(100, 2048)
        )
      )
      (list_embedding_1): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.05, inplace=False)
          (encoder): Embedding(300, 100)
          (rnn): LSTM(100, 2048)
        )
      )
    )
    (word_reprojection_map): Linear(in_features=4096, out_features=256, bias=True)
    (rnn): GRU(256, 256, batch_first=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Linear(in_features=256, out_features=2, bias=True)
  (dropout): Dropout(p=0.0, inplace=False)
  (locked_dropout): LockedDropout(p=0

/Users/huiningonn/anaconda3/envs/nlp_project/lib/python3.8/site-packages/flair/trainers/trainer.py:499: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=use_amp and flair.device.type != "cpu")


2024-12-04 18:54:17,774 epoch 1 - iter 1/7 - loss 0.69338685 - time (sec): 21.70 - samples/sec: 0.37 - lr: 0.000000 - momentum: 0.000000
2024-12-04 18:54:34,932 epoch 1 - iter 2/7 - loss 0.71438470 - time (sec): 38.85 - samples/sec: 0.41 - lr: 0.000025 - momentum: 0.000000
2024-12-04 18:54:50,813 epoch 1 - iter 3/7 - loss 0.70975542 - time (sec): 54.74 - samples/sec: 0.44 - lr: 0.000050 - momentum: 0.000000
2024-12-04 18:55:03,183 epoch 1 - iter 4/7 - loss 0.69591980 - time (sec): 67.10 - samples/sec: 0.48 - lr: 0.000048 - momentum: 0.000000
2024-12-04 18:55:08,366 epoch 1 - iter 5/7 - loss 0.69491612 - time (sec): 72.29 - samples/sec: 0.55 - lr: 0.000045 - momentum: 0.000000
2024-12-04 18:55:16,929 epoch 1 - iter 6/7 - loss 0.69524315 - time (sec): 80.85 - samples/sec: 0.59 - lr: 0.000043 - momentum: 0.000000
2024-12-04 18:55:37,075 epoch 1 - iter 7/7 - loss 0.68840644 - time (sec): 101.00 - samples/sec: 0.54 - lr: 0.000040 - momentum: 0.000000
2024-12-04 18:55:37,076 ----------------

100%|██████████| 2/2 [00:28<00:00, 14.15s/it]

2024-12-04 18:56:05,563 DEV : loss 0.6889662742614746 - f1-score (micro avg)  0.52
2024-12-04 18:56:05,598 saving best model


...

2024-12-04 19:00:24,754 DEV : loss 0.686703085899353 - f1-score (micro avg)  0.56
2024-12-04 19:00:24,789 saving best model


2024-12-04 19:00:25,076 ----------------------------------------------------------------------------------------------------
2024-12-04 19:00:25,077 Loading model from best epoch ...


100%|██████████| 3/3 [01:05<00:00, 21.76s/it]

2024-12-04 19:01:30,524 
Results:
- F-score (micro) 0.5
- F-score (macro) 0.395
- Accuracy 0.5

By class:
              precision    recall  f1-score   support

         NEG     0.5238    0.8462    0.6471        26
         POS     0.3333    0.0909    0.1429        22

    accuracy                         0.5000        48
   macro avg     0.4286    0.4685    0.3950        48
weighted avg     0.4365    0.5000    0.4160        48

2024-12-04 19:01:30,524 ----------------------------------------------------------------------------------------------------


{'test_score': 0.5}

In [9]:
# Path to the saved model
saved_model_path = "flair_flair_small_test_model/best-model.pt"  # Update this if the path or filename is different

# Load the trained model
flair_small_classifier = TextClassifier.load(saved_model_path).to(flair.device)

# Evaluate the model
results = evaluate_model(flair_small_classifier, small_corpus.test)

Evaluating: 100%|██████████| 48/48 [01:33<00:00,  1.94s/it]

True Labels Sample: ['NEG', 'NEG', 'NEG', 'NEG', 'NEG']
Predicted Labels Sample: ['NEG', 'NEG', 'NEG', 'NEG', 'NEG']
True Label Distribution: Counter({'NEG': 26, 'POS': 22})
Predicted Label Distribution: Counter({'NEG': 42, 'POS': 6})
Mapped True Labels Sample: [0, 0, 0, 0, 0]
Mapped Predicted Labels Sample: [0, 0, 0, 0, 0]

Accuracy: 0.5000
Precision: 0.3333
Recall: 0.0909
F1 Score: 0.1429

Classification Report:
              precision    recall  f1-score   support

         NEG       0.52      0.85      0.65        26
         POS       0.33      0.09      0.14        22

    accuracy                           0.50        48
   macro avg       0.43      0.47      0.39        48
weighted avg       0.44      0.50      0.42        48



## flair embedding: actual training

In [12]:
# Set the logging level to INFO
logging.basicConfig(level=logging.INFO)

# Initialize the trainer
trainer = ModelTrainer(flair_classifier, corpus)

#Fine-tune the model
trainer.fine_tune(
    base_path='flair_transformer_model',             # Directory to save the model and logs
    learning_rate=5e-5,                  # Learning rate for fine-tuning
    mini_batch_size=8,                   # Smaller batch size for transformers
    max_epochs=5,                        # Number of epochs
    embeddings_storage_mode='gpu',     
    optimizer=AdamW,                     # Optimizer suited for transformers
    save_final_model=True,               # Save the final model
    save_model_each_k_epochs=1,          # Save model checkpoint every epoch
    create_file_logs=True,               # Save logs to a file
    create_loss_file=True,               # Save loss values to a file
    use_final_model_for_eval=False
)

2024-12-04 19:23:03,117 ----------------------------------------------------------------------------------------------------
2024-12-04 19:23:03,118 Model: "TextClassifier(
  (embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.05, inplace=False)
          (encoder): Embedding(300, 100)
          (rnn): LSTM(100, 2048)
        )
      )
      (list_embedding_1): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.05, inplace=False)
          (encoder): Embedding(300, 100)
          (rnn): LSTM(100, 2048)
        )
      )
    )
    (word_reprojection_map): Linear(in_features=4096, out_features=256, bias=True)
    (rnn): GRU(256, 256, batch_first=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Linear(in_features=256, out_features=2, bias=True)
  (dropout): Dropout(p=0.0, inplace=False)
  (locked_dropout): LockedDropout(p=0

100%|██████████| 157/157 [24:14<00:00,  9.26s/it]

2024-12-04 20:32:49,182 DEV : loss 0.5398534536361694 - f1-score (micro avg)  0.7296


2024-12-04 20:32:55,899 saving best model
2024-12-04 20:32:56,044 ----------------------------------------------------------------------------------------------------
2024-12-04 20:43:00,450 epoch 2 - iter 281/2813 - loss 0.56741641 - time (sec): 604.41 - samples/sec: 3.72 - lr: 0.000049 - momentum: 0.000000
2024-12-04 20:50:01,202 epoch 2 - iter 562/2813 - loss 0.56086464 - time (sec): 1025.16 - samples/sec: 4.39 - lr: 0.000049 - momentum: 0.000000
2024-12-04 20:56:53,066 epoch 2 - iter 843/2813 - loss 0.54737712 - time (sec): 1437.02 - samples/sec: 4.69 - lr: 0.000048 - momentum: 0.000000
2024-12-04 21:03:19,606 epoch 2 - iter 1124/2813 - loss 0.54221408 - time (sec): 1823.56 - samples/sec: 4.93 - lr: 0.000048 - momentum: 0.000000
2024-12-04 21:08:58,776 epoch 2 - iter 1405/2813 - loss 0.54341581 - time (sec): 2162.73 - samples/sec: 5.20 - lr: 0.000047 - momentum: 0.000000
2024-12-04 21:14:56,166 epoch 2 - iter 1686/2813 - loss 0.53647269 - time (sec): 2520.12 - samples/sec: 5.35 - l

100%|██████████| 157/157 [30:57<00:00, 11.83s/it] 

...

100%|██████████| 157/157 [06:55<00:00,  2.65s/it]

2024-12-05 01:39:53,353 DEV : loss 0.4273413419723511 - f1-score (micro avg)  0.7988


2024-12-05 01:39:59,715 ----------------------------------------------------------------------------------------------------
2024-12-05 01:40:03,464 ----------------------------------------------------------------------------------------------------
2024-12-05 01:40:03,464 Exiting from training early.
2024-12-05 01:40:03,465 Saving model ...
2024-12-05 01:40:03,678 Done.
2024-12-05 01:40:03,679 ----------------------------------------------------------------------------------------------------
2024-12-05 01:40:03,679 Loading model from best epoch ...


  1%|          | 11/1563 [00:15<36:29,  1.41s/it]


KeyboardInterrupt: 

In [ ]:
# Path to the saved model
saved_model_path = "flair_transformer_model/best-model.pt"  # Update this if the path or filename is different

# Load the trained model
flair_classifier = TextClassifier.load(saved_model_path).to(flair.device)

# Evaluate the model
results = evaluate_model(flair_classifier, corpus.test)

## transformer classifier: to evaluate if code is working on small dataset

In [7]:
# Initialize transformer embeddings
transformer_embedding = TransformerDocumentEmbeddings(
    model="roberta-base",  # Transformer model of choice
    fine_tune=True,        # Fine-tune the transformer model
    layers="-1",           # Use the last layer for representation
)

# Update classifier with transformer embedding
transformer_classifier = TextClassifier(embeddings=transformer_embedding, label_dictionary=label_dict, label_type="sentiment").to(flair.device)

In [12]:
# Set the logging level to INFO
logging.basicConfig(level=logging.INFO)

# Initialize the trainer
trainer = ModelTrainer(transformer_classifier, small_corpus)

#Fine-tune the model
trainer.fine_tune(
    base_path='flair_transformer_small_test_model',             # Directory to save the model and logs
    learning_rate=5e-5,                  # Learning rate for fine-tuning
    mini_batch_size=8,                   # Smaller batch size for transformers
    max_epochs=2,                        # Number of epochs
    embeddings_storage_mode='gpu',     
    optimizer=AdamW,                     # Optimizer suited for transformers
    save_final_model=True,               # Save the final model
    save_model_each_k_epochs=1,          # Save model checkpoint every epoch
    create_file_logs=True,               # Save logs to a file
    create_loss_file=True,               # Save loss values to a file
    use_final_model_for_eval=False
)


2024-12-04 19:10:38,394 ----------------------------------------------------------------------------------------------------
2024-12-04 19:10:38,395 Model: "TextClassifier(
  (embeddings): TransformerDocumentEmbeddings(
    (model): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(50266, 768)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0-11): 12 x RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True

/Users/huiningonn/anaconda3/envs/nlp_project/lib/python3.8/site-packages/flair/trainers/trainer.py:499: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=use_amp and flair.device.type != "cpu")


2024-12-04 19:10:44,409 epoch 1 - iter 1/7 - loss 1.04363453 - time (sec): 6.00 - samples/sec: 1.33 - lr: 0.000000 - momentum: 0.000000
2024-12-04 19:10:47,945 epoch 1 - iter 2/7 - loss 0.90796769 - time (sec): 9.54 - samples/sec: 1.68 - lr: 0.000050 - momentum: 0.000000
2024-12-04 19:10:53,608 epoch 1 - iter 3/7 - loss 0.79996492 - time (sec): 15.20 - samples/sec: 1.58 - lr: 0.000046 - momentum: 0.000000
2024-12-04 19:10:57,539 epoch 1 - iter 4/7 - loss 0.77897805 - time (sec): 19.13 - samples/sec: 1.67 - lr: 0.000043 - momentum: 0.000000
2024-12-04 19:11:03,182 epoch 1 - iter 5/7 - loss 0.77820672 - time (sec): 24.77 - samples/sec: 1.61 - lr: 0.000039 - momentum: 0.000000
2024-12-04 19:11:08,492 epoch 1 - iter 6/7 - loss 0.79851161 - time (sec): 30.09 - samples/sec: 1.60 - lr: 0.000036 - momentum: 0.000000
2024-12-04 19:11:12,155 epoch 1 - iter 7/7 - loss 0.79783030 - time (sec): 33.75 - samples/sec: 1.63 - lr: 0.000032 - momentum: 0.000000
2024-12-04 19:11:12,156 -------------------

100%|██████████| 2/2 [00:03<00:00,  1.88s/it]

2024-12-04 19:11:16,578 DEV : loss 0.6676883697509766 - f1-score (micro avg)  0.64
2024-12-04 19:11:16,612 saving best model


...

2024-12-04 19:11:56,426 DEV : loss 0.6994249224662781 - f1-score (micro avg)  0.36


2024-12-04 19:11:57,119 ----------------------------------------------------------------------------------------------------
2024-12-04 19:11:57,120 Loading model from best epoch ...


100%|██████████| 3/3 [00:06<00:00,  2.30s/it]

2024-12-04 19:12:06,212 
Results:
- F-score (micro) 0.5625
- F-score (macro) 0.3996
- Accuracy 0.5625

By class:
              precision    recall  f1-score   support

         NEG     0.5532    1.0000    0.7123        26
         POS     1.0000    0.0455    0.0870        22

    accuracy                         0.5625        48
   macro avg     0.7766    0.5227    0.3996        48
weighted avg     0.7580    0.5625    0.4257        48

2024-12-04 19:12:06,212 ----------------------------------------------------------------------------------------------------


{'test_score': 0.5625}

In [13]:
# Path to the saved model
saved_model_path = "flair_transformer_small_test_model/best-model.pt"  # Update this if the path or filename is different

# Load the trained model
transformer_small_classifier = TextClassifier.load(saved_model_path).to(flair.device)

# Evaluate the model
results = evaluate_model(transformer_small_classifier, small_corpus.test)

Evaluating: 100%|██████████| 48/48 [00:03<00:00, 13.25it/s]

True Labels Sample: ['NEG', 'NEG', 'NEG', 'NEG', 'NEG']
Predicted Labels Sample: ['NEG', 'NEG', 'NEG', 'NEG', 'NEG']
True Label Distribution: Counter({'NEG': 26, 'POS': 22})
Predicted Label Distribution: Counter({'NEG': 47, 'POS': 1})
Mapped True Labels Sample: [0, 0, 0, 0, 0]
Mapped Predicted Labels Sample: [0, 0, 0, 0, 0]

Accuracy: 0.5625
Precision: 1.0000
Recall: 0.0455
F1 Score: 0.0870

Classification Report:
              precision    recall  f1-score   support

         NEG       0.55      1.00      0.71        26
         POS       1.00      0.05      0.09        22

    accuracy                           0.56        48
   macro avg       0.78      0.52      0.40        48
weighted avg       0.76      0.56      0.43        48



## actual training

In [7]:
# Set the logging level to INFO
logging.basicConfig(level=logging.INFO)

# Initialize the trainer
trainer = ModelTrainer(transformer_classifier, corpus)

#Fine-tune the model
trainer.fine_tune(
    base_path='flair_roberta_transformer_model',             # Directory to save the model and logs
    learning_rate=5e-5,                  # Learning rate for fine-tuning
    mini_batch_size=8,                   # Smaller batch size for transformers
    max_epochs=5,                        # Number of epochs
    embeddings_storage_mode='gpu',     
    optimizer=AdamW,                     # Optimizer suited for transformers
    save_final_model=True,               # Save the final model
    save_model_each_k_epochs=1,          # Save model checkpoint every epoch
    create_file_logs=True,               # Save logs to a file
    create_loss_file=True,               # Save loss values to a file
    use_final_model_for_eval=False
)


2024-12-05 10:54:03,697 ----------------------------------------------------------------------------------------------------
2024-12-05 10:54:03,698 Model: "TextClassifier(
  (embeddings): TransformerDocumentEmbeddings(
    (model): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(50266, 768, padding_idx=1)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0-11): 12 x RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSdpaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_feat

C:\Users\cheng\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\transformers\models\roberta\modeling_roberta.py:370: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


2024-12-05 10:54:52,777 epoch 1 - iter 281/2813 - loss 0.55673593 - time (sec): 49.07 - samples/sec: 45.82 - lr: 0.000010 - momentum: 0.000000
2024-12-05 10:55:40,834 epoch 1 - iter 562/2813 - loss 0.48512997 - time (sec): 97.12 - samples/sec: 46.29 - lr: 0.000020 - momentum: 0.000000
2024-12-05 10:56:28,486 epoch 1 - iter 843/2813 - loss 0.46255406 - time (sec): 144.78 - samples/sec: 46.58 - lr: 0.000030 - momentum: 0.000000
2024-12-05 10:57:16,357 epoch 1 - iter 1124/2813 - loss 0.44995912 - time (sec): 192.65 - samples/sec: 46.68 - lr: 0.000040 - momentum: 0.000000
2024-12-05 10:58:04,398 epoch 1 - iter 1405/2813 - loss 0.43937684 - time (sec): 240.69 - samples/sec: 46.70 - lr: 0.000050 - momentum: 0.000000
2024-12-05 10:58:52,167 epoch 1 - iter 1686/2813 - loss 0.43399941 - time (sec): 288.46 - samples/sec: 46.76 - lr: 0.000049 - momentum: 0.000000
2024-12-05 10:59:39,828 epoch 1 - iter 1967/2813 - loss 0.43466675 - time (sec): 336.12 - samples/sec: 46.82 - lr: 0.000048 - momentum:

100%|██████████| 157/157 [00:25<00:00,  6.13it/s]

2024-12-05 11:02:27,397 DEV : loss 0.257555216550827 - f1-score (micro avg)  0.92


...

2024-12-05 11:36:24,884 DEV : loss 0.4084957540035248 - f1-score (micro avg)  0.9412


2024-12-05 11:36:32,431 ----------------------------------------------------------------------------------------------------
2024-12-05 11:36:32,433 Loading model from best epoch ...


100%|██████████| 1563/1563 [04:15<00:00,  6.13it/s]


2024-12-05 11:40:51,445 
Results:
- F-score (micro) 0.9429
- F-score (macro) 0.9429
- Accuracy 0.9429

By class:
              precision    recall  f1-score   support

         POS     0.9317    0.9559    0.9437     12500
         NEG     0.9547    0.9299    0.9422     12500

    accuracy                         0.9429     25000
   macro avg     0.9432    0.9429    0.9429     25000
weighted avg     0.9432    0.9429    0.9429     25000

2024-12-05 11:40:51,446 ----------------------------------------------------------------------------------------------------


{'test_score': 0.94292}

In [8]:
# This is for the 10 epochs

# Set the logging level to INFO
logging.basicConfig(level=logging.INFO)

# Initialize the trainer
trainer = ModelTrainer(transformer_classifier, corpus)

#Fine-tune the model
trainer.fine_tune(
    base_path='flair_roberta_transformer_10_model',             # Directory to save the model and logs
    learning_rate=5e-5,                  # Learning rate for fine-tuning
    mini_batch_size=8,                   # Smaller batch size for transformers
    max_epochs=10,                        # Number of epochs
    embeddings_storage_mode='gpu',     
    optimizer=AdamW,                     # Optimizer suited for transformers
    save_final_model=True,               # Save the final model
    save_model_each_k_epochs=1,          # Save model checkpoint every epoch
    create_file_logs=True,               # Save logs to a file
    create_loss_file=True,               # Save loss values to a file
    use_final_model_for_eval=False
)


2024-12-05 11:43:51,397 ----------------------------------------------------------------------------------------------------
2024-12-05 11:43:51,398 Model: "TextClassifier(
  (embeddings): TransformerDocumentEmbeddings(
    (model): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(50266, 768, padding_idx=1)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0-11): 12 x RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSdpaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_feat

C:\Users\cheng\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\transformers\models\roberta\modeling_roberta.py:370: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


2024-12-05 11:44:39,316 epoch 1 - iter 281/2813 - loss 0.58373917 - time (sec): 47.91 - samples/sec: 46.92 - lr: 0.000005 - momentum: 0.000000
2024-12-05 11:45:27,015 epoch 1 - iter 562/2813 - loss 0.47220784 - time (sec): 95.61 - samples/sec: 47.03 - lr: 0.000010 - momentum: 0.000000
2024-12-05 11:46:14,462 epoch 1 - iter 843/2813 - loss 0.44223183 - time (sec): 143.05 - samples/sec: 47.14 - lr: 0.000015 - momentum: 0.000000
2024-12-05 11:47:02,329 epoch 1 - iter 1124/2813 - loss 0.42621069 - time (sec): 190.92 - samples/sec: 47.10 - lr: 0.000020 - momentum: 0.000000
2024-12-05 11:47:50,144 epoch 1 - iter 1405/2813 - loss 0.41301586 - time (sec): 238.74 - samples/sec: 47.08 - lr: 0.000025 - momentum: 0.000000
2024-12-05 11:48:38,050 epoch 1 - iter 1686/2813 - loss 0.40821100 - time (sec): 286.64 - samples/sec: 47.06 - lr: 0.000030 - momentum: 0.000000
2024-12-05 11:49:25,752 epoch 1 - iter 1967/2813 - loss 0.40049963 - time (sec): 334.34 - samples/sec: 47.07 - lr: 0.000035 - momentum:

100%|██████████| 157/157 [00:26<00:00,  6.00it/s]

2024-12-05 11:52:15,965 DEV : loss 0.7565402388572693 - f1-score (micro avg)  0.8396


...

2024-12-05 13:07:07,265 DEV : loss 0.4370265603065491 - f1-score (micro avg)  0.9456


2024-12-05 13:07:13,963 saving best model
2024-12-05 13:07:14,951 ----------------------------------------------------------------------------------------------------
2024-12-05 13:07:14,953 Loading model from best epoch ...


100%|██████████| 1563/1563 [04:13<00:00,  6.15it/s]


2024-12-05 13:11:31,168 
Results:
- F-score (micro) 0.9432
- F-score (macro) 0.9432
- Accuracy 0.9432

By class:
              precision    recall  f1-score   support

         POS     0.9408    0.9460    0.9434     12500
         NEG     0.9457    0.9405    0.9431     12500

    accuracy                         0.9432     25000
   macro avg     0.9433    0.9432    0.9432     25000
weighted avg     0.9433    0.9432    0.9432     25000

2024-12-05 13:11:31,168 ----------------------------------------------------------------------------------------------------


{'test_score': 0.94324}

In [ ]:
# Path to the saved model
saved_model_path = "flair_roberta_transformer_model/best-model.pt"  # Update this if the path or filename is different

# Load the trained model
transformer_classifier = TextClassifier.load(saved_model_path).to(flair.device)

# Evaluate the model
results = evaluate_model(transformer_classifier, corpus.test)